<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/pytorch-lightning-in-practice/pytorch-lightning/episode_3_pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Episode 3:PyTorch Lightning 

**Reference:**

https://www.youtube.com/watch?v=DbESHcCoWbM&list=PLKMvSHzKWRgLTDtAl3txcULbHiKvQTx6U&index=3

In [ ]:
!pip install pytorch-lightning

In [16]:
import torch

from torch import nn
from torch import optim
from torchvision import datasets, transforms 
from torch.utils.data import random_split, DataLoader

import pytorch_lightning as pl
from torchmetrics.functional import accuracy

import pdb

In [3]:
torch.randn(5).cuda()

tensor([-0.5433,  0.2639,  0.9706, -1.7390, -0.4148], device='cuda:0')

##Lightning

1. model
2. optimizer
3. data
4. training loop "the training magic"
5. validation loop "the validation magic"

In [37]:
class ResNet(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)

    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 + h1) # resudual connection
    logits = self.l3(do)
    return logits

  def configure_optimizers(self):
    optimizer = optim.SGD(self.parameters(), lr=1e-2)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch

    # x: b x 1 x 28 x 28 (B*C*W*H)
    b = x.size(0)
    x = x.view(b, -1)

    # Step 1: forward
    logits = self(x)  # logits

    # Step 2: compute the objective function
    loss = self.loss(logits, y)
    acc = accuracy(logits, y)
    pbar = {"train_acc": acc}

    return {"loss": loss, "progress_bar": pbar}
  
  def validation_step(self, batch, batch_idx):
    results = self.training_step(batch, batch_idx)
    results["progress_bar"]["val_acc"] = results["progress_bar"]["train_acc"]
    del results["progress_bar"]["train_acc"]
    return results

  def validation_epoch_end(self, val_step_outputs):
    # [results, results, results, results, ...]
    avg_val_loss = torch.tensor([x["loss"] for x in val_step_outputs]).mean()
    avg_val_acc = torch.tensor([x["progress_bar"]["val_acc"] for x in val_step_outputs]).mean()
    pbar = {"avg_val_acc": avg_val_acc}
    return {"val_loss": avg_val_loss, "progress_bar": pbar}

  """
  def prepare_data(self):
    datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())

  def setup(self):
    dataset = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
    self.train, self.val = random_split(dataset, [55000, 5000])
  """

  def train_dataloader(self):
    dataset = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
    self.train, self.val = random_split(dataset, [55000, 5000])
    train_loader = DataLoader(self.train, batch_size=32)
    return train_loader

  def val_dataloader(self):
    val_loader = DataLoader(self.val, batch_size=32)
    return val_loader

model = ResNet()

In [38]:
trainer = pl.Trainer(max_epochs=5, accelerator='gpu', devices=1)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type             | Params
------------------------------------------
0 | l1   | Linear           | 50.2 K
1 | l2   | Linear           | 4.2 K 
2 | l3   | Linear           | 650   
3 | do   | Dropout          | 0     
4 | loss | CrossEntropyLoss | 0     
------------------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params
0.220     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

AttributeError: ignored

In [21]:
!ls lightning_logs/

version_0  version_1


In [22]:
!ls lightning_logs/version_1/

checkpoints  events.out.tfevents.1660211717.1227e1e7ce31.76.1  hparams.yaml


In [23]:
!ls lightning_logs/version_1/checkpoints/

'epoch=9-step=18750.ckpt'


In [6]:
class ImageClassifier(nn.Module):
  def __init__(self):
      super().__init__()

In [7]:
# define optimizer
params = model.parameters()
optimizer = optim.SGD(params, lr=1e-2)

In [8]:
# define loss
loss = nn.CrossEntropyLoss()

In [9]:
# define data loader
train_data = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000])

train_loader = DataLoader(train, batch_size=8)
val_loader = DataLoader(val, batch_size=8)

In [11]:
# define training and validation step
epochs = 10

for epoch in range(epochs):
  # training loop
  losses = list()
  accuracies = list()
  model.train()  # set training mode because I use dropout
  for batch in train_loader:
    x, y = batch

    # x: b x 1 x 28 x 28 (B*C*W*H)
    b = x.size(0)
    x = x.view(b, -1).cuda()

    # Step 1: forward
    l = model.cuda()(x)  # l: logits
    # pdb.set_trace()

    # Step 2: compute the objective function
    J = loss(l, y.cuda())

    # Step 3: cleaining the gradients
    model.zero_grad()
    # optimizer.zero_grad()
    # params.grad._zero()

    # Step 4: accumulate the partial derivative of loss wrt params
    J.backward()
    # params.grad.sum_(dL/dparams)

    # Step 5: step in the opposite direction of the gradient
    optimizer.step()
    # with torch.no_grad(): params = params - eta * params.grad

    losses.append(J.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

  print(f"Epoch: {epoch + 1}", end=", ")
  print(f"training loss: \t{torch.tensor(losses).mean():.2f}", end=", ")
  print(f"training accuracy: \t{torch.tensor(accuracies).mean():.2f}")

  # validation loop
  losses = list()
  accuracies = list()
  model.train()  # validation mode because I use dropout
  for batch in val_loader:
    x, y = batch

    # x: b x 1 x 28 x 28 (B*C*W*H)
    b = x.size(0)
    x = x.view(b, -1).cuda()

    # Step 1: forward
    with torch.no_grad():
      l = model(x)

    # Step 2: compute the objective function
    J = loss(l, y.cuda())

    losses.append(J.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

  print(f"Epoch: {epoch + 1}", end=", ")
  print(f"validation loss: \t{torch.tensor(losses).mean():.2f}", end=", ")
  print(f"training accuracy: \t{torch.tensor(accuracies).mean():.2f}")

KeyboardInterrupt: ignored

##Debugging the accuracy

(Pdb) p l.size() >>
torch.Size([32, 10])

(Pdb) p l[0] >>
tensor([-1.2800, -3.7171, -3.1333,  1.7426,  1.8951,  9.9960,  0.2454, -8.6478,
         4.0316, -0.0792], device='cuda:0', grad_fn=<SelectBackward0>)

(Pdb) p l[0].detach().argmax() >>
tensor(5, device='cuda:0')

(Pdb) p l[0].detach().softmax(dim=0) >>
tensor([1.2632e-05, 1.1043e-06, 1.9798e-06, 2.5952e-04, 3.0230e-04, 9.9676e-01,
        5.8069e-05, 7.9746e-09, 2.5602e-03, 4.1974e-05], device='cuda:0')

(Pdb) p [f"{p:.2f}" for p in l[0].detach().softmax(dim=0)] >>
['0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00']

(Pdb) p y.size() >>
torch.Size([32])

(Pdb) p y[:4] >>
tensor([5, 5, 4, 9])

(Pdb) p l.detach().argmax(dim=0)[:4] >>
tensor([ 8, 17,  4,  9], device='cuda:0')

(Pdb) p l.detach().argmax(dim=1)[:4] >>
tensor([5, 8, 4, 9], device='cuda:0')

(Pdb) p y[:4].eq(l.detach().argmax(dim=1).cpu()[:4]) >>
tensor([ True, False,  True,  True])

(Pdb) p y[:4].eq(l.detach().argmax(dim=1).cpu()[:4]).float().mean() >>
tensor(0.7500)

(Pdb) p y.eq(l.detach().argmax(dim=1).cpu()).float().mean() >>
tensor(0.9375)

(Pdb) q